In [1]:
import tensorflow as tf
tf.__version__

2023-03-06 16:23:46.534880: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 16:23:46.903974: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-06 16:23:46.904017: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-06 16:23:48.120567: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

'2.11.0'

In [4]:
dataset = tf.data.TextLineDataset(["gs://mgaiduk-us-central1/ratings/csv_gzip/part000000000000"], compression_type="GZIP")
for line in dataset.batch(16): # batching added to demonstrate that parsing works on higher dimensional tensors
    break
print(line)

tf.Tensor(
[b'70,3948,2,1255219128' b'188,653,2,1025333400'
 b'243,103249,2,1464280162' b'359,3578,2,974700907'
 b'426,1500,2,1371502543' b'440,3022,2,1231472902'
 b'446,2861,2,1017900083' b'626,2162,2,1002897585' b'634,1006,2,865372001'
 b'700,46578,2,1350795469' b'734,3203,2,992217420'
 b'1044,2406,2,944907698' b'1332,1198,2,1529895670'
 b'1367,1037,2,1185835950' b'1409,5530,2,1288302165'
 b'1459,1183,2,889019866'], shape=(16,), dtype=string)


In [5]:
defaults = [tf.constant(0, dtype=tf.int64),
           tf.constant(0, dtype=tf.int64),
           tf.constant(0, dtype=tf.float32),
           tf.constant(0, dtype=tf.int64)]
csv_row = tf.io.decode_csv(line, defaults)
csv_row

[<tf.Tensor: shape=(16,), dtype=int64, numpy=
 array([  70,  188,  243,  359,  426,  440,  446,  626,  634,  700,  734,
        1044, 1332, 1367, 1409, 1459])>,
 <tf.Tensor: shape=(16,), dtype=int64, numpy=
 array([  3948,    653, 103249,   3578,   1500,   3022,   2861,   2162,
          1006,  46578,   3203,   2406,   1198,   1037,   5530,   1183])>,
 <tf.Tensor: shape=(16,), dtype=float32, numpy=
 array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
       dtype=float32)>,
 <tf.Tensor: shape=(16,), dtype=int64, numpy=
 array([1255219128, 1025333400, 1464280162,  974700907, 1371502543,
        1231472902, 1017900083, 1002897585,  865372001, 1350795469,
         992217420,  944907698, 1529895670, 1185835950, 1288302165,
         889019866])>]

In [6]:
@tf.function
def decode_fn(csv_line):
    defaults = [tf.constant(0, dtype=tf.int64),
           tf.constant(0, dtype=tf.int64),
           tf.constant(0, dtype=tf.float32),
           tf.constant(0, dtype=tf.int64)]
    csv_row = tf.io.decode_csv(csv_line, defaults)
    features = {}
    features["userId"] = csv_row[0]
    features["movieId"] = csv_row[1]
    labels = {
        "label": csv_row[2]
    }
    return (features, labels)
dataset = tf.data.TextLineDataset(["gs://mgaiduk-us-central1/ratings/csv_gzip/part000000000000"],
                                  compression_type="GZIP").batch(16, drop_remainder=True).map(decode_fn)
for elem in dataset:
    break
elem # take a look at the resulting data

({'userId': <tf.Tensor: shape=(16,), dtype=int64, numpy=
  array([  70,  188,  243,  359,  426,  440,  446,  626,  634,  700,  734,
         1044, 1332, 1367, 1409, 1459])>,
  'movieId': <tf.Tensor: shape=(16,), dtype=int64, numpy=
  array([  3948,    653, 103249,   3578,   1500,   3022,   2861,   2162,
           1006,  46578,   3203,   2406,   1198,   1037,   5530,   1183])>},
 {'label': <tf.Tensor: shape=(16,), dtype=float32, numpy=
  array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        dtype=float32)>})

In [7]:
filenames = tf.data.Dataset.list_files("gs://mgaiduk-us-central1/ratings/csv_gzip/part*", shuffle=True, seed=42)
dataset = tf.data.TextLineDataset(filenames,
                                  compression_type="GZIP").batch(16, drop_remainder=True).map(decode_fn)
for elem in dataset:
    break
elem

({'userId': <tf.Tensor: shape=(16,), dtype=int64, numpy=
  array([   5,  154,  226,  600,  689,  717,  810,  909, 1208, 1409, 1847,
         2071, 2105, 2177, 2429, 2476])>,
  'movieId': <tf.Tensor: shape=(16,), dtype=int64, numpy=
  array([  191, 69526,   736,   435,  7759,  1240,  2087,  1527,   158,
          1722,  1037,   480,  1261,  6290,    39,  4037])>},
 {'label': <tf.Tensor: shape=(16,), dtype=float32, numpy=
  array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        dtype=float32)>})

In [8]:
ctx = None # will not be none in distributed strategy, see later
def make_dataset_fn(path):
    dataset = tf.data.TextLineDataset([path], compression_type="GZIP")
    dataset = dataset\
        .batch(16, drop_remainder=True)\
    .map(decode_fn)
    return dataset
filenames = tf.data.Dataset.list_files("gs://mgaiduk-us-central1/ratings/csv_gzip/part*", shuffle=True, seed=42)
if ctx and ctx.num_input_pipelines > 1:
    filenames = filenames.shard(ctx.num_input_pipelines, ctx.input_pipeline_id)
dataset = filenames.interleave(make_dataset_fn)
for elem in dataset:
    break
elem

({'userId': <tf.Tensor: shape=(16,), dtype=int64, numpy=
  array([   5,  154,  226,  600,  689,  717,  810,  909, 1208, 1409, 1847,
         2071, 2105, 2177, 2429, 2476])>,
  'movieId': <tf.Tensor: shape=(16,), dtype=int64, numpy=
  array([  191, 69526,   736,   435,  7759,  1240,  2087,  1527,   158,
          1722,  1037,   480,  1261,  6290,    39,  4037])>},
 {'label': <tf.Tensor: shape=(16,), dtype=float32, numpy=
  array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        dtype=float32)>})